# Marine Carbonate System Visualizer

## Interactive Bjerrum Plot for Marine Chemistry

<div style="font-size: 22px; line-height: 1.6;">

<div style="text-align: center; margin: 30px 0;">
    <img src="figures/carbonate_system.svg" width="800" alt="Marine Carbonate System Diagram" style="border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); max-width: 100%;">
</div>

<div style="text-align: center; font-size: 18px; color: #00204C; margin: 20px 0;">
    <strong>Comprehensive visualization of marine carbonate chemistry dynamics</strong>
</div>

</div>

This tool visualizes the marine carbonate system using **Bjerrum plots**. 

**Features:**
- Interactive sliders for Total Alkalinity and DIC
- 4-panel visualization showing carbonate speciation
- Real-time calculations using PyCO2SYS
- Educational tool for marine chemistry courses

**Instructions:**
1. Adjust the sliders below to change Total Alkalinity and DIC values
2. Observe how the carbonate system responds in real-time
3. Use the Reset button to return to default values

## What does this tool show?

**Input parameters:**
- **Total alkalinity** (TA) in μmol/kg
- **Dissolved Inorganic Carbon** (DIC) in μmol/kg

**Fixed conditions:**
- **Salinity** = 35 PSU
- **Temperature** = 25°C

**Visualizations:**
1. **Bjerrum Plot** - Species fractions vs pH
2. **Current Composition** - Pie chart of species
3. **Saturation State** - Aragonite saturation
4. **System Analysis** - Detailed results

## 📚 Marine Carbonate System Fundamentals

### 🔬 What is each parameter?

**🧪 Total Alkalinity (TA)**
- **Definition**: The ocean's capacity to neutralize acids (moles of H⁺ that can be absorbed)
- **Components**: Primarily HCO₃⁻ and CO₃²⁻, but also includes OH⁻, B(OH)₄⁻, HPO₄²⁻, etc.
- **Simplified equation**: TA ≈ [HCO₃⁻] + 2[CO₃²⁻] + [OH⁻] - [H⁺]
- **Ocean range**: 1500-3000 μmol/kg (typical: ~2300 μmol/kg)
- **Behavior**: **Conservative** - doesn't change with temperature/pressure, only with mixing and biology

**🌊 Dissolved Inorganic Carbon (DIC)**  
- **Definition**: Total sum of all dissolved CO₂ forms in seawater
- **Components**: [CO₂*] + [HCO₃⁻] + [CO₃²⁻]
- **Equation**: DIC = [CO₂*] + [HCO₃⁻] + [CO₃²⁻] (where CO₂* = CO₂ + H₂CO₃)
- **Ocean range**: 1000-3000 μmol/kg (typical: ~2020 μmol/kg)
- **Behavior**: **Conservative** - only changes with biological processes and mixing

**🎯 pH (Potential of Hydrogen)**
- **Definition**: Measure of seawater acidity (H⁺ concentration)
- **Scale**: pH = -log₁₀[H⁺] 
- **Ocean range**: 7.5-8.5 (typical: ~8.1)
- **Scales**: Total, Seawater, Free, NBS (we use **Total scale** - recommended for seawater)
- **Behavior**: **Non-conservative** - changes with temperature, pressure, mixing

**💨 pCO₂ (partial pressure of CO₂)**
- **Definition**: Pressure that CO₂ would exert if in equilibrium with the atmosphere
- **Units**: μatm (microatmospheres)
- **Ocean range**: 150-800 μatm (current atmosphere: ~420 μatm)
- **Relationship**: Higher pCO₂ = more CO₂ dissolving into ocean
- **Behavior**: **Non-conservative** - very sensitive to temperature and biology

### ❓ Why use TA and DIC together in this notebook?

#### 🏆 **Scientific Advantages:**

✅ **Conservative parameters**
- Don't change during sample transport or storage
- Ideal for water mass mixing studies
- More stable for laboratory measurements

✅ **Complete system definition**
- With TA and DIC, you can calculate pH, pCO₂, [CO₃²⁻], Ω, etc.
- Represent the two fundamental "dimensions" of the carbonate system
- Following PyCO2SYS convention: `par1_type=1` (TA), `par2_type=2` (DIC)

✅ **Clear biogeochemical interpretation**
- TA: ocean's buffering capacity against acidification
- DIC: total carbon inventory available for organisms

#### 🎓 **Educational Advantages:**

📊 **Complete system visualization**
- Students see how all parameters are interconnected
- Changes in TA/DIC show effects on pH, saturation, speciation

🧠 **Conceptual understanding**
- TA represents ocean's "resistance" to pH changes
- DIC represents "amount" of carbon available for calcifying organisms

🌍 **Real-world applications**
- Ocean acidification studies
- Water mass mixing analysis
- Assessment of biological effects on carbonate chemistry

### 🎯 Learning Objectives

**After completing this activity, students will be able to:**

1. **Explain** the difference between conservative (TA, DIC) and non-conservative (pH, pCO₂) parameters
2. **Interpret** Bjerrum diagrams and predict carbonate speciation
3. **Relate** changes in TA/DIC to effects on calcifying organisms
4. **Evaluate** vulnerability of different marine environments to acidification
5. **Apply** concepts to interpret real marine system data

### 📖 Reference

This tool uses **PyCO2SYS** for all calculations. For detailed information about carbonate system chemistry and calculation methods, see:
- [PyCO2SYS Documentation](https://pyco2sys.readthedocs.io/en/latest/)
- [CO2SYS-MATLAB Guide](https://pyco2sys.readthedocs.io/en/latest/co2sys_nd/)

In [1]:
# load necessary libraries
import PyCO2SYS as pyco2
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, VBox, interactive_output, Button, Layout, HTML
from IPython.display import display

In [2]:
# 🔧 Marine Carbonate System Configuration
# These are the standard seawater conditions for all calculations

CONFIG = {
    'salinity': 35,         # Typical ocean salinity (PSU)
    'temperature': 25,      # Surface temperature (°C)
    'pressure': 0,          # Sea surface pressure (dbar)
    'opt_pH_scale': 1,      # Total pH scale (recommended for seawater)
    'opt_k_carbonic': 10    # Waters et al. (2014) constants 
}

def set_config(**kwargs):
    """Modify calculation conditions for different environments
    
    Examples for students:
    set_config(temperature=15)                # Cold water conditions
    set_config(salinity=30, temperature=10)   # Estuarine conditions
    set_config(opt_k_carbonic=8)             # Use Lueker et al. (2000) constants
    """
    for key, value in kwargs.items():
        if key in CONFIG:
            old_value = CONFIG[key]
            CONFIG[key] = value
            print(f"✓ Changed {key}: {old_value} → {value}")
        else:
            print(f"⚠ Unknown parameter: {key}")
            print(f"📋 Available parameters: {list(CONFIG.keys())}")
    
    print(f"\n🌊 Current seawater conditions: {CONFIG}")

# Educational information about constants
print("🔬 Marine Carbonate System Configuration Ready!")
print("📚 Carbonic acid constants options:")
print("   8  = Lueker et al. (2000) - Widely used")
print("   10 = Waters et al. (2014) - Current best practice ⭐")
print("   14 = Schockman & Byrne (2021) - Latest research")
print("\n💡 Students: Try set_config(temperature=15) to see cold water effects!")

🔬 Marine Carbonate System Configuration Ready!
📚 Carbonic acid constants options:
   8  = Lueker et al. (2000) - Widely used
   10 = Waters et al. (2014) - Current best practice ⭐
   14 = Schockman & Byrne (2021) - Latest research

💡 Students: Try set_config(temperature=15) to see cold water effects!


In [3]:
# ⚗️ Marine Carbonate Chemistry Calculator (TA-DIC Method)

def get_pyco2_params(alkalinity, DIC):
    """Prepare parameters for TA-DIC calculations
    
    This function uses Total Alkalinity and Dissolved Inorganic Carbon
    to calculate all other carbonate system parameters.
    """
    return {
        'par1': alkalinity,         # Input 1: Total Alkalinity (μmol/kg)
        'par2': DIC,               # Input 2: Dissolved Inorganic Carbon (μmol/kg)
        'par1_type': 1,            # Parameter type 1 = Total Alkalinity
        'par2_type': 2,            # Parameter type 2 = DIC
        'salinity': CONFIG['salinity'],
        'temperature': CONFIG['temperature'], 
        'pressure': CONFIG['pressure'],
        'opt_pH_scale': CONFIG['opt_pH_scale'],
        'opt_k_carbonic': CONFIG['opt_k_carbonic']
    }

def compute_carbonate_system(alkalinity, DIC):
    """Calculate complete carbonate system from TA and DIC
    
    Students: This function takes your input values and calculates:
    - pH of the seawater
    - pCO2 (how much CO2 would be in equilibrium with atmosphere)
    - Concentrations of HCO3- and CO3-2
    - Aragonite saturation state (important for shells/corals)
    """
    try:
        # Run the calculation using PyCO2SYS
        results = pyco2.sys(**get_pyco2_params(alkalinity, DIC))
        
        # Extract and organize results for plotting
        return {
            "pH_total": float(results["pH_total"]),
            "pCO2": float(results["pCO2"]), 
            "bicarbonate": float(results["bicarbonate"]),
            "carbonate": float(results["carbonate"]),
            "DIC": float(DIC),  # Input DIC value
            "omega_aragonite": float(results["saturation_aragonite"]),
            "alkalinity": float(alkalinity),  # Input alkalinity value
            **CONFIG
        }
        
    except Exception as e:
        print(f"❌ Calculation Error: {e}")
        print(f"📋 Check your inputs: TA={alkalinity}, DIC={DIC}")
        print("💡 Typical values: TA=2300, DIC=2020 μmol/kg")
        raise

def get_constants_from_current_system(data):
    """Extract equilibrium constants for Bjerrum plot
    
    Students: This calculates K1 and K2 from the current system state
    to draw the species distribution curves correctly.
    """
    try:
        pH = data['pH_total']
        pCO2 = data['pCO2'] 
        HCO3 = data['bicarbonate']
        CO3 = data['carbonate']
        
        # Calculate [H+] and [CO2*] concentrations
        H = 10**(-pH)
        CO2_star = pCO2 * 0.034  # Henry's law constant for CO2
        
        # Calculate equilibrium constants from concentrations
        K1 = (H * HCO3 / CO2_star) if CO2_star > 0 else 1e-6
        K2 = (H * CO3 / HCO3) if HCO3 > 0 else 1e-9
        
        return K1, K2
        
    except Exception as e:
        print(f"⚠ Constants calculation issue: {e}")
        return 1e-6, 1e-9  # Safe fallback values

print("⚗️ TA-DIC calculation functions ready!")
print("📊 This tool uses Total Alkalinity and DIC as inputs")
print("🎯 Perfect for studying complete carbonate system relationships")

⚗️ TA-DIC calculation functions ready!
📊 This tool uses Total Alkalinity and DIC as inputs
🎯 Perfect for studying complete carbonate system relationships


In [ ]:
# 🛡️ Data Validation and Marine Scenarios

def validate_marine_parameters(alkalinity, DIC):
    """Validate that input parameters are within realistic marine ranges
    
    Students: This function helps you avoid unrealistic values that could
    give misleading results or calculation errors.
    
    Reference ranges based on Global Ocean Data Analysis Project (GLODAP)
    and World Ocean Atlas data compilations.
    """
    warnings = []
    errors = []
    
    # Realistic ranges for marine environments (based on global ocean data)
    ranges = {
        'alkalinity': (1500, 3000, "μmol/kg"),
        'DIC': (1000, 3000, "μmol/kg")
    }
    
    # Check alkalinity against realistic marine ranges
    alk_min, alk_max, alk_unit = ranges['alkalinity']
    if alkalinity < alk_min or alkalinity > alk_max:
        if alkalinity < 1000 or alkalinity > 4000:
            errors.append(f"❌ Alkalinity {alkalinity} {alk_unit} is outside possible range (1000-4000)")
            errors.append("   💡 Check: Are you using the right units? (μmol/kg, not mmol/kg)")
        else:
            warnings.append(f"⚠️ Alkalinity {alkalinity} {alk_unit} is outside typical ocean range ({alk_min}-{alk_max})")
            warnings.append("   🌊 This might represent extreme environments (freshwater mixing, evaporation)")
    
    # Check DIC against realistic marine ranges  
    dic_min, dic_max, dic_unit = ranges['DIC']
    if DIC < dic_min or DIC > dic_max:
        if DIC < 500 or DIC > 4000:
            errors.append(f"❌ DIC {DIC} {dic_unit} is outside possible range (500-4000)")
            errors.append("   💡 Check: Are you using the right units? (μmol/kg, not mmol/kg)")
        else:
            warnings.append(f"⚠️ DIC {DIC} {dic_unit} is outside typical ocean range ({dic_min}-{dic_max})")
            warnings.append("   🌊 This might represent extreme biological activity or freshwater influence")
    
    # Check alkalinity-DIC relationship (important for marine systems)
    if alkalinity > 0 and DIC > 0:
        ratio = alkalinity / DIC
        if ratio < 0.8:
            warnings.append(f"⚠️ TA/DIC ratio ({ratio:.2f}) is unusually low for seawater")
            warnings.append("   🧪 This suggests very high CO₂ addition (extreme acidification)")
        elif ratio > 2.0:
            warnings.append(f"⚠️ TA/DIC ratio ({ratio:.2f}) is unusually high for seawater")
            warnings.append("   🧪 This suggests carbonate precipitation or CO₂ degassing")
        # Normal range: 1.0-1.3 for most marine environments
    
    return warnings, errors

def get_marine_scenarios():
    """Get predefined marine scenarios for educational purposes
    
    These scenarios represent realistic marine environments with
    typical TA and DIC values based on oceanographic observations.
    
    Reference: Global Ocean Data Analysis Project (GLODAP) and
    World Ocean Atlas climatological data.
    """
    scenarios = {
        "🌊 Typical Open Ocean": {
            "alkalinity": 2300, "DIC": 2020,
            "description": "Average surface ocean conditions, representative of most marine environments"
        },
        "🧊 Polar Surface Water": {
            "alkalinity": 2280, "DIC": 2180,
            "description": "Cold water holds more CO₂, higher DIC due to temperature effect"
        },
        "🌴 Tropical Surface Water": {
            "alkalinity": 2320, "DIC": 1950,
            "description": "Warm water releases CO₂, lower DIC due to temperature and biological activity"
        },
        "🏭 Acidified Future (2100)": {
            "alkalinity": 2300, "DIC": 2150,
            "description": "Projected ocean chemistry under high CO₂ scenario (1000 ppm atmospheric CO₂)"
        },
        "🏞️ Coastal/Estuarine": {
            "alkalinity": 2100, "DIC": 1950,
            "description": "Lower alkalinity due to freshwater mixing, variable DIC from terrestrial inputs"
        },
        "🌋 Deep Water/Upwelling": {
            "alkalinity": 2350, "DIC": 2250,
            "description": "High DIC from organic matter remineralization, higher alkalinity from depth"
        },
        "🪸 Coral Reef (Day)": {
            "alkalinity": 2280, "DIC": 1980,
            "description": "Photosynthesis removes CO₂, calcification removes TA and DIC"
        },
        "🌙 Coral Reef (Night)": {
            "alkalinity": 2290, "DIC": 2080,
            "description": "Respiration adds CO₂, dissolution adds TA and DIC"
        }
    }
    return scenarios

def apply_scenario(scenario_name, sliders):
    """Apply a predefined scenario to the sliders
    
    Students: Use this to quickly set realistic marine chemistry values
    for different environments.
    """
    scenarios = get_marine_scenarios()
    if scenario_name in scenarios:
        scenario = scenarios[scenario_name]
        sliders['alk'].value = scenario['alkalinity']
        sliders['dic'].value = scenario['DIC']
        print(f"📍 Applied scenario: {scenario_name}")
        print(f"   TA = {scenario['alkalinity']}, DIC = {scenario['DIC']} μmol/kg")
        print(f"   {scenario['description']}")
        
        # Validate the scenario
        warnings, errors = validate_marine_parameters(scenario['alkalinity'], scenario['DIC'])
        if warnings or errors:
            for warning in warnings:
                print(warning)
            for error in errors:
                print(error)
    else:
        print(f"❌ Unknown scenario: {scenario_name}")
        print(f"? Available scenarios: {list(scenarios.keys())}")

print("?🛡️ Data validation and marine scenarios loaded!")
print("📊 Realistic ranges: TA(1500-3000), DIC(1000-3000) μmol/kg")
print("🌍 Eight predefined marine scenarios available for educational exploration")
print("💡 Use apply_scenario('scenario_name', sliders) to quickly test different environments")

🛡️ Data validation and marine scenarios loaded!
📊 Realistic ranges: TA(1500-3000), DIC(1000-3000) μmol/kg
🌍 Marine scenarios available for educational exploration


---

## 🚀 How to Use This Educational Tool

### **👩‍🎓 For Students:**

#### **Step 1: Understand the Setup**
- **Default conditions**: Tropical surface seawater (35 PSU, 25°C)
- **Input parameters**: Total Alkalinity (TA) and Dissolved Inorganic Carbon (DIC)  
- **Why these inputs?** They're conservative - don't change during sample handling
- **Calculations**: All done using PyCO2SYS with Waters et al. (2014) constants

#### **Step 2: Interactive Exploration**
1. **Start with defaults** (TA=2300, DIC=2020 μmol/kg) - typical ocean values
2. **Move sliders slowly** and observe how the system responds
3. **Pay attention to**:
   - How pH changes with DIC/TA
   - Species distribution in the Bjerrum plot
   - Aragonite saturation state (Ω > 1 = favorable for shells)
   - Warning messages for unrealistic values

#### **Step 3: Guided Experiments**
**Experiment A - DIC Effects:**
- Keep TA=2300, change DIC from 1600→2400
- *Question*: How does adding CO₂ (higher DIC) affect pH?

**Experiment B - Buffering:**
- Keep DIC=2020, change TA from 1800→3000  
- *Question*: How does alkalinity "buffer" against pH changes?

**Experiment C - Future Ocean:**
- Try TA=2300, DIC=2150 (high CO₂ future scenario)
- *Question*: What happens to shell-building organisms?

### **👩‍🏫 For Instructors:**

#### **Pedagogical Notes:**
- **Conceptual sequence**: TA-DIC → TA-pCO₂ → TA-pH (foundation → climate → biology)
- **Key learning**: Conservative vs. non-conservative parameters
- **Common misconceptions**: pH and pCO₂ are *calculated*, not measured directly
- **Assessment ideas**: Predict-observe-explain activities with the sliders

#### **Technical Configuration:**
- **Change conditions**: Use `set_config(parameter=value)` in configuration cell
- **Alternative constants**: `set_config(opt_k_carbonic=8)` for Lueker et al. (2000)
- **Different environments**: `set_config(temperature=15, salinity=30)` for estuarine

#### **Learning Objectives Alignment:**
✅ Understand carbonate system equilibria  
✅ Interpret Bjerrum diagrams  
✅ Connect chemistry to biological impacts  
✅ Apply to real-world scenarios  

### **📊 Understanding the Visualizations:**

1. **Bjerrum Plot** (top-left): Shows fraction of DIC in each form vs pH
   - *Red line*: Current system pH
   - *Curves*: CO₂, HCO₃⁻, CO₃²⁻ dominance regions

2. **Composition Pie** (top-right): Current speciation breakdown
   - *Useful for*: Seeing which species dominates at current conditions

3. **Saturation State** (bottom-left): Aragonite saturation (Ω)
   - *Ω > 1*: Favorable for shell/coral formation
   - *Ω < 1*: Shells and corals dissolve

4. **System Analysis** (bottom-right): Detailed numerical results
   - *Input values*: Your chosen TA and DIC
   - *Calculated results*: pH, pCO₂, concentrations
   - *Technical details*: Constants and conditions used

---

In [5]:
# 📊 Plotting Functions - Bjerrum Diagram and Analysis

def create_bjerrum_plot(data):
    """Create 4-panel Bjerrum plot with system analysis"""
    plt.close('all')
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Marine Carbonate System Analysis', fontsize=16, fontweight='bold')
    
    colors = ['#443983', '#31688e', '#35b779', '#fde725']
    pH_range = np.linspace(4, 11, 40)
    
    # Get constants from current system state
    K1, K2 = get_constants_from_current_system(data)
    
    # Calculate speciation using analytical expressions
    H_values = 10**(-pH_range)
    denom = H_values**2 + K1*H_values + K1*K2
    
    alpha0 = H_values**2 / denom      # CO2
    alpha1 = K1*H_values / denom      # HCO3-
    alpha2 = K1*K2 / denom            # CO3-2
    
    # Plot 1: Bjerrum diagram
    ax1.plot(pH_range, alpha0, color=colors[0], linewidth=3, label='CO2')
    ax1.plot(pH_range, alpha1, color=colors[1], linewidth=3, label='HCO3-')
    ax1.plot(pH_range, alpha2, color=colors[2], linewidth=3, label='CO3-2')
    
    current_pH = data['pH_total']
    ax1.axvline(x=current_pH, color='red', linestyle='--', linewidth=2)
    ax1.text(current_pH + 0.1, 0.8, f'pH: {current_pH:.2f}', fontsize=10, color='red')
    ax1.set_xlabel('pH'), ax1.set_ylabel('Fraction of DIC')
    ax1.set_title('Bjerrum Plot'), ax1.legend(), ax1.grid(True, alpha=0.3)
    ax1.set_xlim(4, 11), ax1.set_ylim(0, 1)
    
    # Plot 2: Composition pie chart
    CO2_conc = data["pCO2"] * 0.034
    species = [CO2_conc, data["bicarbonate"], data["carbonate"]]
    ax2.pie(species, labels=['CO2', 'HCO3-', 'CO3-2'], autopct='%1.1f%%', colors=colors[:3])
    ax2.set_title(f'Composition at pH {current_pH:.2f}')
    
    # Plot 3: Saturation state
    omega = data["omega_aragonite"]
    ax3.bar(['Aragonite'], [omega], color=colors[3] if omega >= 1 else colors[1])
    ax3.axhline(y=1, color='black', linestyle='--')
    ax3.set_ylabel('Omega'), ax3.set_title('Saturation State')
    ax3.text(0, omega*0.7, f'Ω = {omega:.2f}', ha='center', fontsize=12)
    
    # Plot 4: System info
    ax4.axis('off')
    TA = data['bicarbonate'] + 2*data['carbonate']
    DIC = sum(species)
    
    info_text = f"""Input: TA={TA:.0f}, DIC={DIC:.0f} μmol/kg
    
Results: pH={current_pH:.2f}, pCO2={data['pCO2']:.0f} μatm
[HCO3-]={data['bicarbonate']:.0f}, [CO3-2]={data['carbonate']:.0f} μmol/kg
Ω_arag={omega:.2f}

Constants: K1={K1:.2e}, K2={K2:.2e}
Conditions: S={data['salinity']}, T={data['temperature']}°C
PyCO2SYS: pH_scale={data['opt_pH_scale']}, K_carbonic={data['opt_k_carbonic']}"""
    
    ax4.text(0.05, 0.95, info_text, transform=ax4.transAxes, fontsize=9,
             verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.show()
    return fig

print("📊 Plotting functions ready!")

📊 Plotting functions ready!


In [6]:
# 🎛️ Interactive Interface - Clean and Simple

# Create sliders
sliders = {
    'alk': FloatSlider(value=2300, min=1800, max=3500, step=50,
                      description="Alkalinity (μmol/kg):", 
                      style={'description_width': 'initial'}, layout=Layout(width='600px')),
    'dic': FloatSlider(value=2020, min=1600, max=3500, step=50,
                      description="DIC (μmol/kg):",
                      style={'description_width': 'initial'}, layout=Layout(width='600px'))
}

# Update function (no duplicates)
def update_plots(alkalinity, DIC):
    """Update plot when sliders change"""
    data = compute_carbonate_system(alkalinity, DIC)
    create_bjerrum_plot(data)

# Reset function
def reset_values(b):
    """Reset sliders to default values"""
    sliders['alk'].value, sliders['dic'].value = 2300, 2020

# Reset button
reset_button = Button(description="🔄 Reset Values", button_style='info')
reset_button.on_click(reset_values)

# Display complete interface
display(VBox([
    HTML("<h3>🌊 Marine Carbonate System Explorer</h3>"),
    HTML("<p><em>📊 Complete 4-panel visualization using PyCO2SYS</em></p>"),

    sliders['alk'], 
    sliders['dic'], 
    reset_button,
    interactive_output(update_plots, {'alkalinity': sliders['alk'], 'DIC': sliders['dic']})
]))

print("🎯 Interactive carbonate system interface ready!")
print("📋 All calculations use PyCO2SYS with your chosen constants")
print("💡 Modify CONFIG above to change calculation parameters")

🎯 Interactive carbonate system interface ready!
📋 All calculations use PyCO2SYS with your chosen constants
💡 Modify CONFIG above to change calculation parameters


**Author:** Cardoso-Mohedano JG  
**Institution:** Instituto de Ciencias del Mar y Limnologia, UNAM, Estacion El Carmen  
**License:** [CC BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)  
**ORCID:** [0000-0002-2918-972X](https://orcid.org/0000-0002-2918-972X)

---
This interactive tool allows exploration of the marine carbonate system using [PyCO2SYS](https://pyco2sys.readthedocs.io/en/latest/).  
Developed with support from [Claude AI](https://claude.ai) by Anthropic and [OpenAI ChatGPT](https://openai.com/chatgpt) and educational Python tools.